In [1]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numba
from my_stuff import MufexKeys
from numba.experimental import jitclass
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.utils import clear_cache


# from nb_quantfreedom.nb_base import backtest_df_only
# from nb_quantfreedom.nb_enums import *
# from nb_quantfreedom.nb_helper_funcs import nb_dos_cart_product
# from nb_quantfreedom.strategies.nb_strategy import nb_BacktestInd, nb_Strategy
# from nb_quantfreedom.nb_custom_logger import nb_PrintLogs
# from nb_quantfreedom.nb_order_handler.nb_class_helpers import nb_GetMinPrice


# clear_cache()

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
global_vars_list = [
    ("price_tick_step", numba.types.int_),
    ("market_fee_pct", numba.types.float_),
]


class TestSLClass:
    def __init__(
        self,
        price_tick_step,
        market_fee_pct,
    ):
        self.price_tick_step = price_tick_step
        self.market_fee_pct = market_fee_pct

    def calc_sl_callback(self, callback):
        pass

    def calc_sl(self, price, lookback, thinger):
        pass


@jitclass(global_vars_list)
class SLCalc(TestSLClass):
    def calc_sl_callback(self, callback):
        return callback(self)

    def calc_sl(self, price, lookback, thinger):
        return self.price_tick_step * self.market_fee_pct * price - thinger / lookback


@jitclass(global_vars_list)
class newSLCals(TestSLClass):
    def calc_sl_callback(self, callback):
        return callback(self)

    def calc_sl(self, price, lookback, thinger):
        return self.price_tick_step * self.market_fee_pct * price / lookback / thinger


@numba.njit(cache=True)
def nb_simulate(sl_calculator: TestSLClass):
    return sl_calculator.calc_sl(price=10, lookback=5, thinger=6)


@numba.njit(cache=True)
def nb_base(sl_calculator: TestSLClass):
    return sl_calculator.calc_sl_callback(nb_simulate)

sl_calculator = SLCalc(1, 10.5)
nb_base(sl_calculator=sl_calculator)

103.8